# Introduction

You are an urban safety planner in Japan, and you are analyzing which areas of Japan are in need of extra earthquake reinforcement.  Which areas are both high in population density and prone to earthquakes?

<center>
<img src="https://i.imgur.com/Kuh9gPj.png" width="450"><br/>
</center>

Before you get started, run the code cell below to set everything up.

In [ ]:
import pandas as pd
import geopandas as gpd

import folium
from folium import Choropleth
from folium.plugins import HeatMap

You'll use the `embed_map()` function from the tutorial to visualize your maps.

In [ ]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Exercises

### 1) Do earthquakes coincide with plate boundaries?

Run the code cell below to create a GeoDataFrame `plate_boundaries` that shows global plate boundaries.  The "coordinates" column is a list of (latitude, longitude) locations along the edges of the boundaries.

In [ ]:
plate_boundaries = gpd.read_file("../input/geospatial-course-data/Plate_Boundaries/Plate_Boundaries.shp")
plate_boundaries['coordinates'] = plate_boundaries.apply(lambda x: [(b,a) for (a,b) in list(x.geometry.coords)], axis='columns')
plate_boundaries.drop('geometry', axis=1, inplace=True)

plate_boundaries.head()

Next, run the code cell below without changes to load the historical earthquake data into a DataFrame `earthquakes`.

In [ ]:
# Load the data and print the first 5 rows
earthquakes = pd.read_csv("../input/geospatial-course-data/earthquakes1970-2014.csv", parse_dates=["DateTime"])
earthquakes.head()

The code cell below visualizes the plate boundaries on a map.  Use all of the earthquake data to add a heatmap to the same map, to determine whether earthquakes coincide with plate boundaries.  

In [ ]:
# Create a base map with plate boundaries
m_1 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_1)

# Your code here: Add a heatmap to the map
HeatMap(data=earthquakes[['Latitude', 'Longitude', 'Magnitude']], radius=15, blur=10).add_to(m_1)

embed_map(m_1, 'q_1.html')

So, given the map above, do earthquakes coincide with plate boundaries?

In [ ]:
#q_1.hint()
#q_1.solution()

### 2) Is there a relationship between earthquake depth and proximity to a plate boundary in Japan?

You recently read that the depth of earthquakes tells us [important information about the structure of the earth](https://www.usgs.gov/faqs/what-depth-do-earthquakes-occur-what-significance-depth?qt-news_science_products=0#qt-news_science_products).  You're interested to see if there are any intereresting global patterns, and you'd also like to understand how depth varies in Japan.



In [ ]:
# Create a base map with plate boundaries
m_2 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_2)
    
# Your code here: Add a map to visualize earthquake depth
def color_producer(val):
    if val < 50:
        return 'forestgreen'
    elif val < 100:
        return 'darkorange'
    else:
        return 'darkred'

for i in range(0,len(earthquakes)):
    folium.Circle(
        location=[earthquakes.iloc[i]['Latitude'], earthquakes.iloc[i]['Longitude']],
        radius=2000,
        color=color_producer(earthquakes.iloc[i]['Depth'])).add_to(m_2)

embed_map(m_2, 'q_2.html')

Can you detect a relationship between proximity to a plate boundary and earthquake depth?  Does this pattern hold globally?  In Japan?

In [ ]:
#q_2.hint()
#q_2.solution()

### 3) Which prefectures have high population density?

Run the next code cell (without changes) to create a GeoDataFrame `prefectures` that contains the geographical boundaries of Japanese prefectures.

In [ ]:
# GeoDataFrame with prefecture boundaries
prefectures = gpd.read_file("../input/geospatial-course-data/japan-prefecture-boundaries/japan-prefecture-boundaries.shp")
prefectures.set_index('prefecture', inplace=True)
prefectures.head()

The next code cell creates a DataFrame `stats` containing the population, area (in square kilometers), and population density (per square kilometer) for each Japanese prefecture.  Run the code cell without changes.

In [ ]:
# DataFrame containing population of each prefecture
population = pd.read_csv("../input/geospatial-course-data/japan-prefecture-population.csv")
population.set_index('prefecture', inplace=True)

# Calculate area (in square kilometers) of each prefecture
area_sqkm = pd.Series(prefectures.geometry.to_crs(epsg=3035).area / 10**6, name='area_sqkm')
stats = population.join(area_sqkm)

# Add density (per square kilometer) of each prefecture
stats['density'] = stats["population"] / stats["area_sqkm"]
stats.head()

Use the next code cell to create a choropleth map to visualize population density.

In [ ]:
geo_json_data = prefectures['geometry'].__geo_interface__
plot_dict = stats['density']

m_3 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

Choropleth(
    geo_data=geo_json_data,
    data=plot_dict,
    key_on="feature.id",
    fill_color='YlGnBu',
    legend_name='Population density (per square kilometer)').add_to(m_3)

embed_map(m_3, 'q_3.html')

Which prefectures have relatively higher density than the others?  Are they spread throughout the country, or all located in roughly the same geographical region?

In [ ]:
#q_3.hint()
#q_3.solution()

### 4) Which high-density prefectures are prone to high-magnitude earthquakes?

Create a map to suggest prefectures that might benefit from earthquake reinforcement.  Your map should visualize both density and earthquake magnitude.

In [ ]:
m_4 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

def color_producer(magnitude):
    if magnitude > 6.5:
        return 'red'
    else:
        return 'green'

Choropleth(
    geo_data=geo_json_data,
    data=plot_dict,
    key_on="feature.id",
    fill_color='BuPu',
    legend_name='Population density (per square kilometer)').add_to(m_4)

for i in range(0,len(earthquakes)):
    folium.Circle(
        location=[earthquakes.iloc[i]['Latitude'], earthquakes.iloc[i]['Longitude']],
        popup=("{} ({})").format(
            earthquakes.iloc[i]['Magnitude'],
            earthquakes.iloc[i]['DateTime'].year),
        radius=earthquakes.iloc[i]['Magnitude']**5.5,
        color=color_producer(earthquakes.iloc[i]['Magnitude'])).add_to(m_4)

embed_map(m_4, 'q_4.html')

Which prefectures of Japan do you recommend for extra earthquake reinforcement?

In [ ]:
#q_4.hint()
#q_4.solution()

# Keep going

continue to learn about ...